Precision: Calculating how many of the extracted relationships are valid

In [2]:
!pip install deepeval


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import nltk
from nltk.stem import PorterStemmer
nltk.download("punkt")

# Initialize Python porter stemmer
ps = PorterStemmer()
def lemmatize(sent):
    return [ps.stem(word) for word in sent.split()]

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rishikasrinivas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
import fitz,os
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

#reading the pdf and hand annotation files
def read_pdf(pdf_file):
    start=False
    sentences=[]
    start_idx=0
    with fitz.open(pdf_file) as pdf_file:
        for page_index, page in enumerate(pdf_file):
            text = page.get_text("text").lower()
            text=text.split(". ")
            sentences.extend(text)
                
    return sentences
def read_files(root_dir, hand):
    
    lines=[]
    for files in os.listdir(root_dir):
        if files[-4:] != '.pdf':
            continue
        sentences = read_pdf(f"{root_dir}/{files}")
        lines.extend(sentences)

    # read in hand annotations
    for p in hand.iterrows():
        rel = p[1]['rel']
        subj = p[1]['subj']
        obj = p[1]['obj']
        out=f"{subj} {rel} {obj}" 
        lines.append(out)


    return lines

#computing cosine similarity
def vec(sentences):
    # Encode sentences
    embeddings = model.encode([sentences[0], sentences[1]])
    
    # Compute cosine similarity
    similarity = util.cos_sim(embeddings[0], embeddings[1])
    return similarity.item() # Value close to 1 indicates high similarity
    
#finding if the target string (relation triplet) is in the src (pdf + hand annotation)
def find(target, src):
    found=False
    matching_sentence=""
 
    for idx,sentence in enumerate(src):
        pred=" ".join(lemmatize(target))
        test=" ".join(lemmatize(sentence))
        cos = vec([pred,test])
        if pred in test or cos > 0.7:
            if cos >0.65 and cos < 0.7:
                print(f"Got a match for {pred }: {sentence}")
            elif cos <=0.65:
                print(f"Closest match to {pred} was {test}")
            found=True
            st_idx=idx
            matching_sentence=sentence
            return found, matching_sentence, 
            
    return found, matching_sentence


In [5]:
print(f"Example of a sentence from the pdf: {sentences[4]}\n\n")
print(f"Example from the hand annot. {sentences[-1]}")

NameError: name 'sentences' is not defined

In [6]:
#find target in hand annotations
def find_in_hand(hand,target):
    max_=-1.0
    for row in hand.iterrows():
        hand_rel = f" {row[1]['subj']} {row[1]['rel']} {row[1]['obj']}"
        sub_h = row[1]['subj']
        cos=vec([hand_rel, target])
        

        if cos > max_:
            max_ = cos
            best=hand_rel
        
        cos=vec([best, hand_rel])
        if sub_h.lower() not in best.lower() or cos < 0.7:
            print(f"Closest match for {hand_rel} is {best} with {cos}")
            continue
        else:
            return best
            #print(f"Closest match for {hand_rel} is {best} with {vec([best, hand_rel])}")
            #print(rouge_([hand_rel], [best]))
            
    
            

In [ ]:
from deepeval.metrics import ContextualRecallMetric
from deepeval.test_case import LLMTestCase
import pandas as pd
hand = pd.read_csv("../Results/ground_truth.csv")
pred_files = ["../NewRels_Skip3_PassingInIncrements.csv","../NewRels_Skip4_increments.csv", "../Results/NewRels_Skip3_cummulative.csv", "../Results/NewRels_Skip2_cummulative.csv"]
sentences = read_files("../Docs",hand)


for pred_file in pred_files[1:]:
    preds = pd.read_csv(pred_file) 
    score = 0
    for p in preds.iterrows():
        ref = p[1]['ref']
        rel = p[1]['rel']
        subj = p[1]['subj']
        obj = p[1]['obj']
        out=f"{subj} {rel} {obj}"
        
        found, match = find(out, sentences)
        if found:
            score +=1
        else:
            print("Couldn't find a match for  ", out)
    print(f"Precision for {pred_file} is {score/len(preds)}")

Couldn't find a match for   ketamine not effective as a miracle drug for all depression cases
Couldn't find a match for   cognitive behavioral therapy (cbt) first-line treatment mild to moderate major depressive disorder
Couldn't find a match for   interpersonal therapy (ipt) first-line treatment mild to moderate major depressive disorder
Couldn't find a match for   venlafaxine side effects nausea, vomiting, and sexual dysfunction
Couldn't find a match for   physical exercise definition an adjunct therapy recommended for alleviating depression symptoms and preventing relapse, improving quality of life
Couldn't find a match for   vagus nerve stimulation (vns) definition therapeutic method for resistant unilateral or bipolar depression
Couldn't find a match for   magnetic seizure therapy comparable to ect
Couldn't find a match for   phototherapy (luxtherapy) specific efficacy in treating seasonal and non-seasonal depression
Couldn't find a match for   citalopram more effective reboxetine

In [ ]:
print(f"Recall score: {100*score/len(preds)}%")

In [ ]:
Precision for ../NewRels_Skip3_PassingInIncrements.csv : 0.8181818181818182
Precision for ../NewRels_Skip4_increments.csv is 0.6785714285714286